In [28]:
import apriltag
import cv2
import numpy as np

In [29]:
def CreateUI():
	UIHeight = 1500
	UIWidth = 1000

	UIBG = np.zeros((UIHeight,UIWidth,3),dtype=np.uint8)

	BoxHeight = 1000
	BoxWidth = 200
	Gap = 100

	YStart = int((UIHeight-BoxHeight)/2)

	for i in range(3):
		XStart = Gap + i*(BoxWidth+Gap)

		BoxTopLeft = (XStart,YStart)
		BoxBottomRight = (XStart+BoxWidth , YStart+BoxHeight)

		cv2.rectangle(UIBG,BoxTopLeft,BoxBottomRight,(192,242,30),-1)
	return UIBG

UIBG = CreateUI()

options = apriltag.DetectorOptions(families="tag16h5")
detector = apriltag.Detector(options)
cam = cv2.VideoCapture(0,cv2.CAP_V4L2)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
if not cam.isOpened():
	print("Cannot Open Camera")
	exit()
Angle = 0

In [30]:
while True:
	success,frame = cam.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	results = detector.detect(gray)

	if results:
		
		Tag = results[0]
		MarkerCorners = Tag.corners

		MarkerTopLeft,MarkerTopRight,MarkerBottomRight,MarkerBottomLeft = MarkerCorners

		TopVector = MarkerTopRight - MarkerTopLeft
		LeftVector = MarkerBottomLeft - MarkerTopLeft

		OffsetScale = 0
		UIScale = 5

		UITopLeft = MarkerBottomLeft+(LeftVector*OffsetScale)
		UITopRight = UITopLeft+(TopVector*UIScale)
		UIBottomLeft = UITopLeft+(LeftVector*UIScale)
		UIBottomRight = UITopLeft+((TopVector+LeftVector)*UIScale)

		DestinationPoints = np.array([UITopLeft,UITopRight,UIBottomRight,UIBottomLeft], dtype=np.float32)

		h,w,c = UIBG.shape

		SourcePoints = np.array([[0,0],[w,0],[w,h],[0,h]], dtype=np.float32)

		Matrix,_ = cv2.findHomography(SourcePoints,DestinationPoints)

		if Matrix is not None:
			WarpedUI = cv2.warpPerspective(UIBG, Matrix, (frame.shape[1],frame.shape[0]))

			mask = np.sum(WarpedUI, axis=2) > 0
			frame[mask] = WarpedUI[mask]

	cv2.imshow("VideoFeed",frame)

	if cv2.waitKey(1) == ord('q'):
		break

cam.release()
cv2.destroyAllWindows()